In [21]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.stats import poisson
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from scipy.stats import norm
from IPython.display import display


# gum bleeding before using the probiotic
gum_bleeding_BeforePB = pd.read_csv('/students/2023-2024/master/micromouth_marvels/working/Gum-Bleeding.csv')
numerical_gum_bleeding_BeforePB = gum_bleeding_BeforePB.set_index('Weeks').applymap(lambda x: 1 if x == 'Yes' else 0)
total_gum_bleeding_bf_counts = numerical_gum_bleeding_BeforePB.sum()
total_days_observed_bf = numerical_gum_bleeding_BeforePB.shape[0]
proportion_of_days_with_gum_bleeding_bf = total_gum_bleeding_bf_counts / total_days_observed_bf

gum_bleeding_BeforePB_stats_df = pd.DataFrame({
    'Total Gum Bleeding Count Before PB': total_gum_bleeding_bf_counts,
    'Proportion of Days with Gum Bleeding Before PB': proportion_of_days_with_gum_bleeding_bf
})

print(gum_bleeding_BeforePB_stats_df)

# gum bleeding after using the probiotic
gum_bleeding_afterPB = pd.read_csv('/students/2023-2024/master/micromouth_marvels/working/Gum bleeding 2.csv')
numerical_gum_bleeding_afterPB = gum_bleeding_afterPB.set_index('Weeks').applymap(lambda x: 1 if x == 'Yes' else 0)
total_gum_bleeding_af_counts = numerical_gum_bleeding_afterPB.sum()
total_days_observed_af = numerical_gum_bleeding_afterPB.shape[0]
proportion_of_days_with_gum_bleeding_af = total_gum_bleeding_af_counts / total_days_observed_af

gum_bleeding_afterPB_stats_df = pd.DataFrame({
    'Total Gum Bleeding Count After PB': total_gum_bleeding_af_counts,
    'Proportion of Days with Gum Bleeding After PB': proportion_of_days_with_gum_bleeding_af
})

print(gum_bleeding_afterPB_stats_df)


               Total Gum Bleeding Count Before PB  \
participant 1                                   6   
participant 2                                   2   
participant 3                                   4   
participant 4                                   7   
participant 5                                   2   

               Proportion of Days with Gum Bleeding Before PB  
participant 1                                          0.3750  
participant 2                                          0.1250  
participant 3                                          0.2500  
participant 4                                          0.4375  
participant 5                                          0.1250  
               Total Gum Bleeding Count After PB  \
participant 1                                  3   
participant 2                                  2   
participant 3                                  2   
participant 4                                  2   
participant 5                        

/tmp/ipykernel_2045696/201447593.py:14: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  numerical_gum_bleeding_BeforePB = gum_bleeding_BeforePB.set_index('Weeks').applymap(lambda x: 1 if x == 'Yes' else 0)
/tmp/ipykernel_2045696/201447593.py:28: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  numerical_gum_bleeding_afterPB = gum_bleeding_afterPB.set_index('Weeks').applymap(lambda x: 1 if x == 'Yes' else 0)


In [22]:
def DS_2sample_ztest_counts(y1, y2, alternative ='greater', alpha=0.05, method='normal'):
    
    import numpy as np
    from scipy.stats import norm
    from scipy.stats import poisson
    
    n_1 = len(y1) ; n_2 = len(y2)
    lambda_1_ML = np.mean(y1); lambda_2_ML = np.mean(y2)
    # Normal approximation:
    lambda_p = (n_1*lambda_1_ML + n_2*lambda_2_ML)/(n_1 + n_2) # pooled value
    z = (lambda_1_ML - lambda_2_ML)/np.sqrt( lambda_p * (1/n_1 + 1/n_2) )
    
    print(80*'-')
    print('2-sample z-test for lambda:')
    print('     assuming Poisson(lambda.1) data for dataset 1')
    print('     assuming Poisson(lambda.2) data for dataset 2')
    print('lambda.1.ML = {:.3g}, lambda.2.ML = {:.3g}, n.1 = {:d}, n.2 = {:d}, alpha = {:.3g}'.format(lambda_1_ML, lambda_2_ML, n_1, n_2, alpha))
    print('H0: lambda.1  = lambda.2')
    
    if alternative == 'two-sided':
        print('H1: lambda.1 != lambda.2')
        # Normal approximation:
        p_value_N = 2 * norm.cdf(-np.abs(z), 0, 1)
        z_crit_L = norm.ppf(alpha/2, 0, 1)
        z_crit_R = norm.ppf(1-alpha/2, 0, 1)
    elif alternative == 'less':
        print('H1: lambda.1  < lambda.2')
        # Normal approximation:
        p_value_N = norm.cdf(z)
        z_crit_L = norm.ppf(alpha, 0, 1)
        z_crit_R = float('inf')
    elif alternative == 'greater':
        print('H1: lambda.1  > lambda.2')
        # Normal approxumation:
        p_value_N = 1 - norm.cdf(z, 0, 1)
        # better precision, use the survival function:
        p_value_N = norm.sf(z, 0, 1)
        z_crit_L = float('-inf')
        z_crit_R = norm.ppf(1-alpha, 0, 1)
    else:
        print('Wrong alternative hypothesis chosen!')
        print(80*'-' + '\n')
        z, p_value, z_crit_L, z_crit_R = np.nan, np.nan, np.nan, np.nan
        return(z, p_value_N, z_crit_L, z_crit_R)
    
    # Effect size (Cramer's V2):   # Tentative...
    V2 = z**2 * (1/n_1 + 1/n_2)
    # Choose the p-value according to 'method':
    if method == 'norm':
        p_value = p_value_N
    else:
        p_value = p_value_N
    # To fix later:
    #print('z = {:.4g}, p-value = {:.4g} (normal), z.crit.L = {:.4g}, z.crit.R = {:.4g}'.format(z, p_value_N, z_crit_L, z_crit_R))
    print('z = {:.4g}, p-value = {:.4g}, z.crit.L = {:.4g}, z.crit.R = {:.4g}'.format(z, p_value_N, z_crit_L, z_crit_R))
    #print('Effect size: V2 = {:.3g}; benchmarks V2: 0.01 = small, 0.09 = medium, 0.25 = large (???)'.format(V2))
    print(80*'-' + '\n')
    return(z, p_value, z_crit_L, z_crit_R)

In [23]:


def plot_z_test_results(z_value, p_value, z_crit_R):
    plt.figure(figsize=(10, 6))
    x = np.linspace(-3, 3, 1000)
    y = norm.pdf(x, 0, 1)
    
    plt.plot(x, y, label='Standard Normal Distribution')
    plt.scatter(z_value, norm.pdf(z_value), color='red', s=50, label=f'z-value: {z_value:.2f}')
    
    if z_crit_R != float('inf'):
        x_crit = np.linspace(z_crit_R, 3, 1000)
        y_crit = norm.pdf(x_crit, 0, 1)
        plt.fill_between(x_crit, y_crit, color='gray', alpha=0.5, label='Critical Region (alpha=0.05)')
        plt.axvline(x=z_crit_R, color='gray', linestyle='--', label=f'Critical Value: {z_crit_R:.2f}')
    
    plt.title(f'2-Sample Z-Test Visualization\nz-value: {z_value:.2f}, p-value: {p_value:.4f}')
    plt.xlabel('z-value')
    plt.ylabel('Probability Density')
    plt.legend()
    plt.grid(True)
    plt.show()

def interactive_z_test_and_plot(participant_index):
    count_before = total_gum_bleeding_bf_counts[participant_index]
    count_after = total_gum_bleeding_af_counts[participant_index]
    
    counts_before = np.ones(count_before, dtype=int)
    counts_after = np.ones(count_after, dtype=int)
    
    z_value, p_value, z_crit_L, z_crit_R = DS_2sample_ztest_counts(counts_before, counts_after, alternative='greater')
    
    plot_z_test_results(z_value, p_value, z_crit_R)

# Create a dropdown for participant selection
participant_selector = widgets.Dropdown(
    options=[('Participant ' + str(i+1), i) for i in range(len(total_gum_bleeding_bf_counts))],
    value=0,
    description='Select Participant:',
)

# Bind the interactive z-test function to the participant selector
interactive_plot = widgets.interactive(interactive_z_test_and_plot, participant_index=participant_selector)

# Display the interactive elements along with the plot
display(interactive_plot)


interactive(children=(Dropdown(description='Select Participant:', options=(('Participant 1', 0), ('Participant…